<a href="https://colab.research.google.com/github/hadilalshehri/Hadeel/blob/master/MHProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from itertools import islice
import numpy as np
import random

class Problem:

    global no_elements , no_clusters , cluster_type , cluster_limit , node_w, distance_matrix
   # def __init__(self):
      #  self.solu = Solution()

    def GetData(self, file):
        with open(file) as f:
            for line in f:
                line = line.split(" ")
                space_index = [i for i, e in enumerate(line) if e == '']
                for index in sorted(space_index, reverse=True):
                    del line[index]
                break_index = [i for i, e in enumerate(line) if e == '\n']
                for index in sorted(break_index, reverse=True):
                    del line[index]
                no_elements = int(line[0])
                no_clusters = int(line[1])
                cluster_type = line[2]
                cluster_limit = list()
                for i in range(no_clusters):
                    if (i%2) == 0:
                        cluster_limit.append(line[i+3:i+5])
                    else:
                        cluster_limit.append(line[i+2:i+4])
                cluster_limit = [[int(num) for num in lst[:]] for lst in cluster_limit]
                w_index = [i for i, e in enumerate(line) if e == 'W']
                node_w = line[w_index[0]+1:]
                for i in range(0, len(node_w)):
                    node_w[i] = int(node_w[i])
                break
    
        distance_matrix = [[0 for i in range(no_elements)] for i in range(no_elements)]
        with open(file) as f:
            for line in islice(f, 1, None):
                line = line.split(" ")
                line[2] = line[2].replace("\n","")
                node_1 = int(line[0])
                node_2 = int(line[1])
                edge_weight = float(line[2])
                distance_matrix[node_1][node_2] = edge_weight

        return no_elements , no_clusters , cluster_type , cluster_limit , node_w, distance_matrix

    def Constraint(self,Cluster,Cluster_Size):
        Lower_limit =  cluster_limit[Cluster][0] 
        Upper_limit =  cluster_limit[Cluster][1]
        #check Constraint 
        if(Cluster_Size <=Lower_limit):# first try to achive the Lowest limit 
            return True  
        elif (Cluster_Size <= Upper_limit):
            return True
        else:
            return False 
    
    def Constraint_2Nodes(self,node1,node2,Cluster1,Cluster2,Clusters_Size):
        n1_weigth = node_w[node1]
        n2_weigth = node_w[node2]
        c1_size=Clusters_Size[Cluster1]
        c2_size=Clusters_Size[Cluster2]
        
        c1_size = c1_size-n1_weigth
        c1_size = c1_size + n2_weigth
        c2_size = c2_size-n2_weigth
        c2_size = c2_size + n1_weigth        

        #check Constraint 
        if(self.Constraint(Cluster1,c1_size) and self.Constraint(Cluster2,c2_size)):
            return True  
        else:
            return False     
    
    def Constraint_1Nodes(self,node,Cluster1,Cluster2,Clusters_Size):
        node_weigth = node_w[node]
        c1_size=Clusters_Size[Cluster1]
        c2_size=Clusters_Size[Cluster2]
        
        c1_size = c1_size-node_weigth
        c2_size = c2_size + node_weigth        

        #check Constraint 
        if(self.Constraint(Cluster1,c1_size) and self.Constraint(Cluster2,c2_size)):
            return True  
        else:
            return False   




class Solution:
    
    def __init__(self,no_clusters,cluster_limit,node_weight,Edge_matrix):
       self.no_clusters = no_clusters
       self.cluster_limit = cluster_limit
       self.node_weight = node_weight
       self.Edge_matrix = Edge_matrix
    
    def Objective_Function(self,Nodes):
        WeightList= []
        Sol_Obj_Function= 0
        for c in range (no_clusters):
            Edge_weight = 0
            Cluster_Nodes = self.Lookup_Cluster_Nodes(c, Nodes) #Get CLuster Nodes 
            for  i, node in enumerate(Cluster_Nodes) :#Rows
                for n in Cluster_Nodes[i:]: #column
                    if(Edge_matrix[node][n]):
                        Edge_weight = Edge_weight + Edge_matrix[node][n]
                
            WeightList.append(Edge_weight)
        #calcualte weight for all solution
        for w in WeightList:
            Sol_Obj_Function =  w + Sol_Obj_Function
        
        return Sol_Obj_Function,WeightList


    
    #Solution Represention 
    def SolutionRepresention(self, No_Elemenet):
        Nodes_vector = [-1] * No_Elemenet
        return Nodes_vector

    #retrun cluster node  
    def Lookup_Cluster_Nodes(self,cluster,Nodes_Vector):
        NodeListonCluster = []
        for N, C in enumerate(Nodes_Vector):
            if(C==cluster):
                NodeListonCluster.append(N)
    
        return NodeListonCluster 
    
    def Avaliable_Cluster(self,Current_node,Nodes_Vector):
        Avaliable_Constraint = []
        Clusters_Size = []
        for c in range (no_clusters):
            Cluster_Size=0
            Cluster_Nodes = self.Lookup_Cluster_Nodes(c, Nodes_Vector) #Get CLuster Nodes 
            #for C_L in range(len(cluster_limit)): # get Capacity Limit 
            Lower_limit =  cluster_limit[c][0] 
            Upper_limit =  cluster_limit[c][1]
            
            for n in Cluster_Nodes: #Calucate Limit 
                Cluster_Size =Cluster_Size+ node_w[n]
                #if(c==no_clusters-1):
                
            Clusters_Size.append(Cluster_Size)# to check the constraient 
            
            Cluster_Size = Cluster_Size+ node_w[Current_node]# calcualte the size of the current node to check constraint after added 
            
            #check Constraint OR use Problem.Constraint_NodeWeight(self,Cluster,Cluster_Size)
            if(Cluster_Size <=Lower_limit):# first try to achive the Lowest limit 
                Avaliable_Constraint.append(c)  
            elif (Cluster_Size <= Upper_limit):
                Avaliable_Constraint.append(c) 

        
        return  Avaliable_Constraint,Clusters_Size
    
    #Check edge 
    def check_Edge(self,node,cluster, Edge_Matrix,Nodes_Vector):
        avaliable_Edge= False
        NodeList_Cluster = self.Lookup_Cluster_Nodes(cluster,Nodes_Vector)
        for n in NodeList_Cluster:
            if(Edge_Matrix[node][n]):
                avaliable_Edge= True
                #listofweigth.append(Edge_Matrix[node][n])# # return list of weigth 
                break
            if(Edge_Matrix[n][node]):
                avaliable_Edge= True
                #listofweigth.append(Edge_Matrix[n][node])# return list of weigth 
                break            
        return avaliable_Edge
    
    def Cluster_NotEmpty(self,C,Nodes):
        for val in Nodes:
            if (val == C):
                return True
  
    def UnAssigned_Node(self, Nodes_Vector):
        list = []
        for n,c in enumerate(Nodes_Vector):
            if(c==-1):
                list.append(n)
        return list 

    #Greedy Hueristic 
    def Greedy_Huerstic(self,Nodes_Vector):
        for n in range(len(Nodes_Vector)):
            ClusterList,a = self.Avaliable_Cluster(n,Nodes_Vector)#  send the current node to check when added the wich size will effected 
            for c in ClusterList:
                if(self.Cluster_NotEmpty(c,Nodes_Vector)):
                    AvaliableEdge = self.check_Edge(n,c,Edge_matrix,Nodes_Vector)
                    if(AvaliableEdge):
                        Nodes_Vector[n]=c
                        break
                else:
                    Nodes_Vector[n]=c
                    break

            if(Nodes_Vector[n] is None and ClusterList != None):
              Nodes_Vector[n]= random.choice(ClusterList)         

        return Nodes_Vector #Initial Solution

    #Greedy Hueristic + Randomisation 
    def Greedy_Huerstic_Rand(self,Nodes_Vector):
        for n in range(len(Nodes_Vector)):
            ClusterList,a = self.Avaliable_Cluster(n,Nodes_Vector)
            for itration in range(12):
                c= random.choice(ClusterList)  
                if(self.Cluster_NotEmpty(c,Nodes_Vector)):
                    AvaliableEdge = self.check_Edge(n,c,Edge_matrix,Nodes_Vector)
                    if(AvaliableEdge):
                        Nodes_Vector[n]=c
                        break
                else:
                    Nodes_Vector[n]=c
                    break

            if(Nodes_Vector[n]is None and ClusterList != None):
              Nodes_Vector[n]= random.choice(ClusterList)         

        return Nodes_Vector #Initial Solution
  
    #Random  Solution  
    def Random_Solution(self,Nodes_Vector):
        while True: # |Stopped when all Nodes assigned 
            unassigned_node = self.UnAssigned_Node(Nodes_Vector)
            if(len(unassigned_node)):
              n = random.choice(unassigned_node)
            else:
                break
            ClusterList ,a= self.Avaliable_Cluster(n,Nodes_Vector)
            #ClusterList = range(12) # for test
            for C_itration in range(len(ClusterList)):
                c= random.choice(ClusterList)  
                if(self.Cluster_NotEmpty(c,Nodes_Vector)):
                    AvaliableEdge = self.check_Edge(n,c,Edge_matrix,Nodes_Vector)
                    if(AvaliableEdge):
                        Nodes_Vector[n]=c
                        ClusterList.remove(c)
                        break
                else:
                    Nodes_Vector[n]=c
                    ClusterList.remove(c)
                    break

            if(Nodes_Vector[n]):
              Nodes_Vector[n]= random.choice(ClusterList)         

        return Nodes_Vector #Initial Solution
              
    def Print_solution(self,Solution):

        for c in range(12):
            cluster=[]
            for n, val in enumerate(Solution):
              if (val == c):
                  cluster.append(n) 
           
            print('Cluster ',c+1,"is",cluster)        

class algorithem:
    #Object
    
    problem = Problem()

    def __init__(self,no_elements, no_clusters,cluster_limit,node_weight,Edge_matrix):
       self.no_elements= no_elements
       self.no_clusters = no_clusters
       self.cluster_limit = cluster_limit
       self.node_weight = node_weight
       self.Edge_matrix = Edge_matrix
       self.sol = Solution(no_clusters,cluster_limit,node_weight,Edge_matrix)
   
    def Neighborhood (self, Solution,clusters_Size):
        Swap_Visited_Solution=[]
        Move_visited_Node = [-1] * no_elements 
        Neighborhood_List  = []
        Operator = []
        #Swap 
        while(True):
            c1,c2 = random.choices(range(no_clusters),2)
            c1_nodes = sol.Lookup_Cluster_Nodes(c1,Solution)
            c2_nodes = sol.Lookup_Cluster_Nodes(c2,Solution)
            n1= random.choice(c1_nodes)
            n2 = random.choice(c2_nodes)
            if(itertion_visited== 100): # To conrtrol the  infint loop 
                break
        #visited Swap
            for i in Swap_Visited_Solution:
                if(Swap_Visited_Solution[i] == [n1,n2] or Swap_Visited_Solution[i] == [n2,n1]):
                     itertion_visited= itertion_visited+1
                     continue
        #Not Visited Swap 
         # Check constraint 
            if(problem.Constraint_2Nodes(n1,n2,c1,c2,clusters_Size)):
                #swapping operator
                Copy_Solution = Solution
                Copy_Solution[n1] = c2
                Copy_Solution[n2] = c1
                new_neigbor = Copy_Solution            
                Neighborhood_List.append(new_neigbor)
                Operator.append(['swap',n1,n2,c1,c2])
                 #Add to Swap visited List 
                Swap_Visited_Solution.append([n1,n2])
        
            
        #move
        iteration=0
        while(iteration < no_elements):

            node = random.choice(range(len(Solution)))
             #visited Move
            if(Move_visited_Node[node] ==-1):
                continue 
             
             #Not visited Move
            C_node = Solution[node]
            #new_cluster = random.choice([x for x in range(no_clusters) if x != C_node])#or use remove C_node
            for new_c in   [x for x in range(no_clusters) if x != C_node]:
                if(problem.Constraint_1Nodes(node,C_node,new_c)):
                    #Move Operator
                    Copy_Solution = Solution
                    Copy_Solution[node] = new_c
                    new_neigbor = Copy_Solution
                    Neighborhood_List.append(new_neigbor)
                    Operator.append(['move',node,-1,C_node,new_c])
            #Add node to  visited List        
            Move_visited_Node[node]=0 
            #Control the loop
            iteration = iteration+1

                

        return Neighborhood_List, Operator #'list of neighborhood solution '    
    
    def local_search():

        return 'optimal solution '
 


X = Problem()    
no_elements , no_clusters , cluster_type , cluster_limit , node_w, Edge_matrix = X.GetData("read.txt")
solution = Solution(no_clusters,cluster_limit,node_w,Edge_matrix)

Nodes = solution.SolutionRepresention(no_elements)
print(Nodes)
#Greedy_Huerstic
Initial_Solution = solution.Greedy_Huerstic(Nodes)
print(Initial_Solution)
solution.Print_solution(Initial_Solution)
OBj,List = solution.Objective_Function(Initial_Solution)
#Greedy Huerstic with Rand
Greedy_Solution = solution.Greedy_Huerstic_Rand(Nodes)
print(Greedy_Solution)
solution.Print_solution(Greedy_Solution)

# Random Solution 
Random_Solution = solution.Random_Solution(Nodes)
print('---------------------------------------------')
solution.Print_solution(Random_Solution)


#check constraint is fit ????
a,Cluster_Size = solution.Avaliable_Cluster(0,Random_Solution)
for  c, size in enumerate(Cluster_Size):
    print('Cluster',c,'Cluster_Size',size)

#objective function for  Solution

OBjG,ListG = solution.Objective_Function(Greedy_Solution)
OBjR,ListR = solution.Objective_Function(Random_Solution)
print('objective function for Initial Solution',OBj)
print('objective function for Greedy Solution',OBjG)
print('objective function for Random Solution',OBjR)
print(List)
print(ListG)
print(ListR)


FileNotFoundError: ignored